### task01 1. 使用豆瓣电影评论数据完成文本分类处理：文本预处理，加载、构建词典。（评论得分1～2	表示positive取值：1，评论得分4～5代表negative取值：0）
### https://www.kaggle.com/datasets/utmhikari/doubanmovieshortcomments

#### 数据预处理

In [ ]:
# 修复后的存盘文件
fixed = open("fixed_comments.csv", "w", encoding="utf-8")

# 修复前的内容文件
#读取
lines = [line for line in open(r"E:\code\VS_code\BADOU_AIsystem_file\week07\task\DMSC.csv", "r", encoding="utf-8")]
#读取

for i ,line in enumerate(lines):#i为索引line为内容
    #enumerate() 是 Python 的一个内置函数，用于在遍历可迭代对象（如列表、字符串、文件行等）时，
    #同时获取元素的索引（序号）和元素本身
    #保存标题列
    if i == 0:
        fixed.write(line)
        pre_line = ''#上一行的书名
        continue
    #提取书名和评论文本
    terms = line.split(',')

    #是否属于同一本书
    if len(terms) != 10: continue
    if pre_line!= '' and terms[2] == pre_line.split(',')[2]:
        if len(pre_line.split(",")) == 10:#上一行的内容是否是一个完整的记录
            #保存上一行的记录
            fixed.write(pre_line + '\n')
            pre_line = line.strip()#去除当前行 (line) 的首尾空白字符（包括换行符 \n 和空格等） #，并将处理后的字符串保存到 pre_line 中
            
        else:
            pre_line =""
    #是否属于不同本书
    else:  
        fixed.write(line)
        pre_line = line.strip()#保存当前行
        

            
            
        
fixed.close()

In [46]:
## 评论得分列表的构建
##所获得的列表为一个包含元组的列表
import csv
import jieba
import pickle
#用户评论集
ds_coments = []
with open(r'E:\code\VS_code\BADOU_AIsystem_file\week07\task\fixed_comments.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)#使用 csv.DictReader（适用于有表头的 CSV）
    for row in reader:
        if len(ds_coments)>50000:#五万条数据
            break
        vote =int(row['Star'])
        
        if vote in [1,2,4,5]:
            words =jieba.lcut(row['Comment'])
            ds_coments.append((words, 0 if vote > 0 and vote < 3 else 1)) 
ds_commnts = [c for c in ds_coments if len(c[0]) in range(1, 150)]


with open(r'E:\code\VS_code\BADOU_AIsystem_file\week07\task\score_list.pkl', 'wb') as f:
    pickle.dump(ds_commnts, f)
print(len(ds_coments) )

50001


In [35]:
print(ds_coments[8])

([' ', '承认', '这货', '很烂', '很难', '吗', '？', '混乱', '的', '节奏', '，', '人人', '都', '可能', '冒出来', '即兴', '插一腿', '，', '糟糕', '的', '3D', '也', '是', '毫无', '必要', '，', '就', '连', '特效', '大', '场面', '都', '没有', '什么', '令人', '惊艳', '的', '地方', '，', '其实', '这些', '都', '不', '重要', '了', '，', '我', '只', '想', '知道', '一点', '，', '黑寡妇', '下', '一部', '的', '时候', '会', '跟', '哪位', '队友', '搞', '暧昧', '？', '这种', '“', '白痴', '专供', '”', '片子', '如今', '算是', '彻底', '找到', '自己', '的', '死忠', '了', '，', '漫威', '可以', '继续', '一招', '鲜吃遍', '天', '了', '。', '★', '★'], 0)


### 加载数据+构建词典

In [36]:
#分词字典的构建
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence  # 长度不同张量填充为相同长度
import jieba
import csv
def build_from_doc(doc):
    vocab = set()
    for line in doc:
        vocab.update(line[0])

    vocab =  ['PAD','UNK'] + list(vocab)  # PAD: padding, UNK: unknown
    w2idx = {word: idx for idx, word in enumerate(vocab)}
    return w2idx



### task02加载处理后文本构建词典、定义模型、训练、评估、测试。

In [38]:
#定义模型
class Comments_Classifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)  # padding_idx=0
        self.rnn = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, input_ids):
        # input_ids: (batch_size, seq_len)
        # embedded: (batch_size, seq_len, embedding_dim)
        embedded = self.embedding(input_ids)
        # output: (batch_size, seq_len, hidden_size)
        output, (hidden, _) = self.rnn(embedded)
        output = self.fc(output[:, -1, :])  # 取最后一个时间步的输出
        return output


In [ ]:
# 训练模型

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle


if __name__ == '__main__':
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 加载训练语料
    with open(r'E:\code\VS_code\BADOU_AIsystem_file\week07\task\score_list.pkl','rb') as f:
        comments_data = pickle.load(f)

    # 构建词汇表
    vocab = build_from_doc(comments_data)
    print('词汇表大小:', len(vocab))

    # 所有向量集合 Embedding（词嵌入）
    emb = nn.Embedding(len(vocab), 100) # 词汇表大小，向量维度

    # 自定义数据转换方法(callback function)回调函数
    # 该函数会在每个batch数据加载时被调用
    def convert_data(batch_data):
        comments, votes = [],[]
        # 分别提取评论和标签
        for comment, vote in batch_data:
            comments.append(torch.tensor([vocab.get(word, vocab['UNK']) for word in comment]))
            votes.append(vote)
        
        # 将评论和标签转换为tensor
        commt = pad_sequence(comments, batch_first=True, padding_value=vocab['PAD'])  # 填充为相同长度
        labels = torch.tensor(votes)
        # 返回评论和标签
        return commt, labels

    # 通过Dataset构建DataLoader
    dataloader = DataLoader(comments_data, batch_size=4, shuffle=True, 
                            collate_fn=convert_data)#collate_fn: 自定义数据转换方法

    # 构建模型
    # vocab_size: 词汇表大小
    # embedding_dim: 词嵌入维度
    # hidden_size: LSTM隐藏层大小
    # num_classes: 分类数量
    vocab_size = len(vocab)
    embedding_dim = 100
    hidden_size = 128
    num_classes = 2

    model = Comments_Classifier(len(vocab), embedding_dim, hidden_size, num_classes)
    model

    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # 训练模型
    num_epochs = 1
    for epoch in range(num_epochs):
        for i, (cmt, lbl) in enumerate(dataloader):
            cmt = cmt
            lbl = lbl

            # 前向传播
            outputs = model(cmt)
            loss = criterion(outputs, lbl)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    # 保存模型
    torch.save(model.state_dict(), 'comments_classifier_se.pth')
    # 模型词典
    torch.save(vocab, 'comments_vocab_se.pth')


词汇表大小: 46039
Epoch [1/1], Step [100/12501], Loss: 0.5558
Epoch [1/1], Step [200/12501], Loss: 0.2539
Epoch [1/1], Step [300/12501], Loss: 0.5538
Epoch [1/1], Step [400/12501], Loss: 0.5475
Epoch [1/1], Step [500/12501], Loss: 0.6152
Epoch [1/1], Step [600/12501], Loss: 0.3533
Epoch [1/1], Step [700/12501], Loss: 0.5667
Epoch [1/1], Step [800/12501], Loss: 0.5232
Epoch [1/1], Step [900/12501], Loss: 0.5347
Epoch [1/1], Step [1000/12501], Loss: 0.6175
Epoch [1/1], Step [1100/12501], Loss: 0.8888
Epoch [1/1], Step [1200/12501], Loss: 0.5534
Epoch [1/1], Step [1300/12501], Loss: 1.2594
Epoch [1/1], Step [1400/12501], Loss: 0.5404
Epoch [1/1], Step [1500/12501], Loss: 1.1500
Epoch [1/1], Step [1600/12501], Loss: 0.4042
Epoch [1/1], Step [1700/12501], Loss: 0.3425
Epoch [1/1], Step [1800/12501], Loss: 0.2997
Epoch [1/1], Step [1900/12501], Loss: 0.8031
Epoch [1/1], Step [2000/12501], Loss: 0.3362
Epoch [1/1], Step [2100/12501], Loss: 0.9432
Epoch [1/1], Step [2200/12501], Loss: 0.5402
Epoch 

In [45]:
# 测试模型
embedding_dim = 100
hidden_size = 128
num_classes = 2

# 加载词典
vocab = torch.load('comments_vocab.pth')
# 测试模型
comment1 = '这部电影很好看，全程无尿点'
comment2 = '没见过这样的烂片，真垃圾'

# 将评论转换为索引
comment1_idx = torch.tensor([vocab.get(word, vocab['UNK']) for word in jieba.lcut(comment1)])
comment2_idx = torch.tensor([vocab.get(word, vocab['UNK']) for word in jieba.lcut(comment2)])
# 将评论转换为tensor
comment1_idx = comment1_idx.unsqueeze(0).to(device)  # 添加batch维度    
comment2_idx = comment2_idx.unsqueeze(0).to(device)  # 添加batch维度

# 加载模型
model = Comments_Classifier(len(vocab), embedding_dim, hidden_size, num_classes)
model.load_state_dict(torch.load('comments_classifier.pth'))
model.to(device)

# 模型推理
pred1 = model(comment1_idx)
pred2 = model(comment2_idx)

# 取最大值的索引作为预测结果
pred1 = torch.argmax(pred1, dim=1).item()
pred2 = torch.argmax(pred2, dim=1).item()
print(f'评论1预测结果: {pred1}')
print(f'评论2预测结果: {pred2}')

评论1预测结果: 1
评论2预测结果: 0


C:\Users\阿尼马\AppData\Local\Temp\ipykernel_896\2934440317.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vocab = torch.load('comments_vocab.pth')
C:\Users\阿尼马\AppData\Lo

### task3. 尝试不同分词工具进行文本分词，观察模型训练结果。

#### 数据处理为sentence可接受的形式

In [ ]:
import sentencepiece as spm

# 训练模型
spm.SentencePieceTrainer.train(
    input=r'/kaggle/working/fixed1_comments.txt',
    model_prefix='m',
    vocab_size=10000,
    character_coverage=1.0,
    model_type='bpe'
)

# 加载模型并构建字典
sp = spm.SentencePieceProcessor()
sp.load('m.model')

vocab_dict = {sp.id_to_piece(i): i for i in range(sp.get_piece_size())}

# 保存字典
import pickle
with open('vocab_dict.pkl', 'wb') as f:
    pickle.dump(vocab_dict, f)

# 打印词汇表大小和前10个token
print(f"词汇表大小: {len(vocab_dict)}")
print("示例token:")
for token, idx in list(vocab_dict.items())[:10]:
    print(f"{idx}: {token}")


词汇表大小: 10000
ID 0: <unk>
ID 1: <s>
ID 2: </s>
ID 3: ..
ID 4: 。。
ID 5: 电影
ID 6: 剧情
ID 7: ....
ID 8: 没有
ID 9: 一部

ID 1对应的token: <s>


#### 训练模型

In [ ]:
# 训练模型

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle


if __name__ == '__main__':
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 加载训练语料
    with open(r'/kaggle/working/score_list.pkl','rb') as f:
        comments_data = pickle.load(f)

    # 构建词汇表
    with open(r'/kaggle/working/vocab_dict.pkl','rb') as f:
        vocab = pickle.load(f)

    # 所有向量集合 Embedding（词嵌入）
    emb = nn.Embedding(len(vocab), 100) # 词汇表大小，向量维度

    # 自定义数据转换方法(callback function)回调函数
    # 该函数会在每个batch数据加载时被调用
    def convert_data(batch_data):
        comments, votes = [],[]
        # 分别提取评论和标签
        for comment, vote in batch_data:
            comments.append(torch.tensor([vocab.get(word, vocab['<unk>']) for word in comment]))
            votes.append(vote)
        
        # 将评论和标签转换为tensor
        commt = pad_sequence(comments, batch_first=True, padding_value=vocab['<s>'])  # 填充为相同长度
        labels = torch.tensor(votes)
        # 返回评论和标签
        return commt, labels

    # 通过Dataset构建DataLoader
    dataloader = DataLoader(comments_data, batch_size=4, shuffle=True, 
                            collate_fn=convert_data)#collate_fn: 自定义数据转换方法

    # 构建模型
    # vocab_size: 词汇表大小
    # embedding_dim: 词嵌入维度
    # hidden_size: LSTM隐藏层大小
    # num_classes: 分类数量
    vocab_size = len(vocab)
    embedding_dim = 100
    hidden_size = 128
    num_classes = 2

    model = Comments_Classifier(len(vocab), embedding_dim, hidden_size, num_classes)
    model.to(device)

    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # 训练模型
    num_epochs = 5
    for epoch in range(num_epochs):
        for i, (cmt, lbl) in enumerate(dataloader):
            cmt = cmt.to(device)
            lbl = lbl.to(device)

            # 前向传播
            outputs = model(cmt)
            loss = criterion(outputs, lbl)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 2000 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    # 保存模型
    torch.save(model.state_dict(), 'comments_classifier.pth')
    # 模型词典
    torch.save(vocab, 'comments_vocab.pth')
    

词汇表大小: 10000
ID 0: <unk>
ID 1: <s>
ID 2: </s>
ID 3: ..
ID 4: 。。
ID 5: 电影
ID 6: 剧情
ID 7: ....
ID 8: 没有
ID 9: 一部

ID 1对应的token: <s>


### 训练过程

![alt text](image-1.png)

#### 测试模型

In [ ]:
# 测试模型
embedding_dim = 100
hidden_size = 128
num_classes = 2

# 加载词典
vocab = torch.load('comments_vocab.pth')
# 测试模型
comment1 = '这部电影很好看，全程无尿点'
comment2 = '真牛，第一次感觉到不好看是什么滋味'

# 将评论转换为索引
comment1_idx = torch.tensor([vocab.get(word, vocab['<unk>']) for word in jieba.lcut(comment1)])
comment2_idx = torch.tensor([vocab.get(word, vocab['<unk>']) for word in jieba.lcut(comment2)])
# 将评论转换为tensor
comment1_idx = comment1_idx.unsqueeze(0).to(device)  # 添加batch维度    
comment2_idx = comment2_idx.unsqueeze(0).to(device)  # 添加batch维度

# 加载模型
model = Comments_Classifier(len(vocab), embedding_dim, hidden_size, num_classes)
model.load_state_dict(torch.load('comments_classifier.pth'))
model.to(device)

# 模型推理
pred1 = model(comment1_idx)
pred2 = model(comment2_idx)

# 取最大值的索引作为预测结果
pred1 = torch.argmax(pred1, dim=1).item()
pred2 = torch.argmax(pred2, dim=1).item()
print(f'评论1预测结果: {pred1}')
print(f'评论2预测结果: {pred2}')

### 测试结果
![alt text](image.png)
